In [6]:
import numpy as np
from collections import deque
import copy
import math
import random
import time


NbClients = 5
clientLocs = []

while len(clientLocs) < NbClients:
    x = random.randint(0, NbClients*10)
    y = random.randint(0, NbClients*10)
    
    # On vérifie si le point (x, y) existe déjà
    if [x, y] not in clientLocs:
        clientLocs.append([x, y])

clientLocs = np.array(clientLocs)

print("Client Locations:\n", clientLocs)

# calcule la matrix des distances entre les clients
def distance_matrix(locs):
    n = locs.shape[0] # Calcul du nombre de clients
    dist_matrix = np.zeros((n, n)) # Création d'une matrix n x n remplie de zéros
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = np.linalg.norm(locs[i] - locs[j]) # Calcul de la distance euclidienne entre les clients i et j
    dist_matrix = np.round(dist_matrix).astype(int) # Arrondi des distances et conversion en entiers

    return dist_matrix

matrix = distance_matrix(clientLocs)
print("Distance Matrix:\n", distance_matrix(clientLocs))


Client Locations:
 [[10 24]
 [13 26]
 [40 23]
 [14 47]
 [42 26]]
Distance Matrix:
 [[ 0  4 30 23 32]
 [ 4  0 27 21 29]
 [30 27  0 35  4]
 [23 21 35  0 35]
 [32 29  4 35  0]]


In [2]:
filename = f"instance/matrix_distances_{NbClients+1}x{NbClients+1}.csv"  
np.savetxt(filename, matrix, delimiter=",", fmt='%d')


In [3]:
depot = 0


def voisinMinPoid(matrix, listeClient, cur):
    poidMinTrajet = 0
    nextVoisin = -1
    
    for i in listeClient:
        if matrix[cur][i] > 0 and poidMinTrajet == 0:
            nextVoisin = i
            poidMinTrajet = matrix[cur][i]
        elif matrix[cur][i] > 0 and matrix[cur][i] < poidMinTrajet:
            poidMinTrajet = matrix[cur][i]
            nextVoisin = i

    return nextVoisin

def voisinsClientGraphematrix(matrix, sommet):
    voisins = [i for i in range(len(matrix)) if matrix[sommet][i] > 0]
    return voisins 

def poidCycle(matrix, cycle):
    poids_total = 0
    for i in range(len(cycle) - 1):
        poids_total += matrix[cycle[i]][cycle[i + 1]]
    poids_total += matrix[cycle[-1]][cycle[0]] 
    return poids_total



def recherche_tabou_cycle(matrix, start, firstNeighbor, iter_max=100):

    # On copie la matrix pour ne pas modifier l’originale
    matrix_copy = copy.deepcopy(matrix)

    # Le cycle que nous construisons (liste d’indices de sommets)
    cycle = [start]

    # Liste tabou : elle garde les derniers sommets visités pour éviter les retours
    tailleTabou = len(matrix) + 5
    tabou = deque(maxlen= tailleTabou)
    tabou.append(start)

    # Le sommet courant (celui où on se trouve actuellement)
    cur = start

    # Boucle principale de la recherche tabou
    for _ in range(iter_max):

        if len(cycle) == 1:
            voisin = firstNeighbor
        else:
            voisins = voisinsClientGraphematrix(matrix_copy, cur) # On récupère la liste des voisins encore connectés du sommet courant
           
            candidats = [i for i in voisins if i not in tabou] # On enlève les voisins qui sont "tabou" 

            # S’il n’y a aucun voisin disponible, on ne peut plus avancer
            if not candidats:
                break

            voisin = voisinMinPoid(matrix_copy, candidats, cur)

        # On retire l’arête entre le sommet courant et le voisin choisi
        matrix_copy[cur][voisin] = 0
        matrix_copy[voisin][cur] = 0
        
        cycle.append(voisin) # On ajoute ce voisin au cycle    
        tabou.append(voisin) # On ajoute le sommet courant dans la liste tabou

        cur = voisin

    # On retourne le chemin (cycle) trouvé
    return cycle



def tabou_multi_start(matrix, nb_lancements=10, iter_max=100):
    """
    Lance plusieurs recherches tabou depuis des sommets de départ aléatoires,
    puis retourne le meilleur cycle (le plus long) trouvé.

    - nb_lancements : nombre d’essais (points de départ différents)
    - iter_max : nombre d’itérations par recherche
    """

    meilleur_cycle = []  # Le meilleur cycle global (le plus court)
    tempsMeilleurCycle = 0
    goodI = 0

    # On répète l’expérience plusieurs fois (multi-start)
    for i in range(nb_lancements):

        start = depot

        firstNeighbor = 0

        while matrix[start][firstNeighbor] == 0:
            firstNeighbor = random.randint(1, len(matrix)-1)

        # On effectue une recherche tabou locale à partir de ce sommet
        cycle = recherche_tabou_cycle(matrix, start, firstNeighbor, iter_max)

        # On affiche le résultat intermédiaire
        print(f"Lancement {i+1}: départ={firstNeighbor}, longueur du cycle={len(cycle)}, temps du trajet={poidCycle(matrix, cycle)}")

        
        if tempsMeilleurCycle == 0:
            tempsMeilleurCycle = poidCycle(matrix, cycle)
            meilleur_cycle = cycle
            goodI = i+1
        elif poidCycle(matrix, cycle) < tempsMeilleurCycle:
            meilleur_cycle = cycle
            tempsMeilleurCycle = poidCycle(matrix, cycle)
            goodI = i+1

    # Après tous les lancements, on renvoie le meilleur
    return meilleur_cycle, tempsMeilleurCycle, goodI


# Mesure du temps d’exécution
start_time = time.time()

print("### Recherche tabou multi-start sur la Zone A ###\n")
print("Nombre de clients :", len(matrix))

# Lancement du multi-start (10 essais, taille tabou = 5, 100 itérations max)
meilleur_cycle, tempsMeilleurCycle, goodI = tabou_multi_start(matrix, 20, 100)

# Fin du chrono
end_time = time.time()
execution_time_ms = (end_time - start_time) * 1000

# Affichage du meilleur résultat trouvé
print("\n=== Meilleur cycle trouvé ===")
print("Lancement n°", goodI, "Longueur du cycle :", len(meilleur_cycle)+ 1, "  Temps du cycle :", tempsMeilleurCycle)
for s in meilleur_cycle:
    print(s + 1, "-> ", end='')
print(meilleur_cycle[0]+1)  # on revient au départ pour fermer le cycle

print("\nTemps d'exécution :", round(execution_time_ms, 2), "ms")


### Recherche tabou multi-start sur la Zone A ###

Nombre de clients : 5
Lancement 1: départ=4, longueur du cycle=5, temps du trajet=117
Lancement 2: départ=1, longueur du cycle=5, temps du trajet=100
Lancement 3: départ=2, longueur du cycle=5, temps du trajet=93
Lancement 4: départ=2, longueur du cycle=5, temps du trajet=93
Lancement 5: départ=4, longueur du cycle=5, temps du trajet=117
Lancement 6: départ=2, longueur du cycle=5, temps du trajet=93
Lancement 7: départ=3, longueur du cycle=5, temps du trajet=92
Lancement 8: départ=4, longueur du cycle=5, temps du trajet=117
Lancement 9: départ=1, longueur du cycle=5, temps du trajet=100
Lancement 10: départ=1, longueur du cycle=5, temps du trajet=100
Lancement 11: départ=2, longueur du cycle=5, temps du trajet=93
Lancement 12: départ=3, longueur du cycle=5, temps du trajet=92
Lancement 13: départ=2, longueur du cycle=5, temps du trajet=93
Lancement 14: départ=1, longueur du cycle=5, temps du trajet=100
Lancement 15: départ=2, longueur 

In [8]:

prob_bouchon = 0.3
facteur = 3



def generer_facteur_bouchon(heure):
    """
    Génère un facteur global de bouchon selon l'heure de la journée.
    - Peu de bouchons la nuit
    - Maximal vers 8h et 17h
    """
    # Heure normalisée sur 24h → sinus pour faire un cycle
    intensite = 0.5 + 0.5 * math.sin((heure - 8) / 24 * 2 * math.pi)
    # Variation entre 1.0 et 3.0 environ
    n = random.uniform(-2,2)
    facteur = n + 2.0 * intensite  
    if facteur <= 0:
        facteur = 1
    return facteur

def cout_effectif(i, j, heure):
    """
    Retourne le coût dynamique entre 2 villes à une heure donnée.
    Complexité O(1)
    """
    base = matrix[i][j]
    if base == 0:
        return 0
    
    # Facteur global du trafic (selon l'heure)
    facteur_temps = generer_facteur_bouchon(heure)
    
    # Petite variation pseudo-aléatoire stable (chaque route a sa propre sensibilité)
    # Cela évite que tout augmente/diminue uniformément
    random.seed(i * 1000 + j)  # stable entre runs
    variation = random.uniform(-0.1, 0.1)  # entre -10% et +10%
    
    cout = base * facteur_temps * (1 + variation)
    return round(cout, 2)

def simulation_journee():
    """
    Simule 24 heures avec bouchons évoluant toutes les 4 heures.
    """
    heures = list(range(0, 25, 4))  # 0h, 4h, 8h, 12h, 16h, 20h, 24h
    for h in heures:
        print(f"\n=== Heure {h}h ===")
        facteur = generer_facteur_bouchon(h)
        print(f"Facteur global de bouchon : {facteur:.2f}")
        
        # Exemple : coût de la route (0 -> 5) évolue dans la journée
        cout_05 = cout_effectif(0, 5, h)
        print(f"Coût entre ville 0 et 5 à {h}h : {cout_05}")


simulation_journee()




=== Heure 0h ===
Facteur global de bouchon : 1.00


IndexError: index 5 is out of bounds for axis 0 with size 5